# 🫀 ECG Cross-Attention Model for Arrhythmia Classification

MIT-BIH Arrhythmia Database를 사용한 ECG 부정맥 분류 모델 학습

**분류 클래스 (AAMI 4-class)**
- N: Normal
- S: SVEB (Supraventricular Ectopic Beat)
- V: VEB (Ventricular Ectopic Beat)
- F: Fusion


## 1️⃣ 환경 설정


In [ ]:
# GPU 확인
!nvidia-smi


In [ ]:
# 필요한 패키지 설치
!pip install -q wfdb openpyxl neurokit2


In [ ]:
# GitHub에서 프로젝트 clone
!git clone https://github.com/h4xryu/ECG_CrossAttention.git
%cd ECG_CrossAttention


## 2️⃣ Google Drive 마운트 & 데이터 연결


In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive 마운트 완료!")



In [ ]:
import os

# Google Drive의 데이터 경로
DRIVE_DATA_PATH = '/content/drive/MyDrive/ECG_Data/mit-bih-arrhythmia-database-1.0.0/'

# 프로젝트 데이터 폴더 생성 및 심볼릭 링크
os.makedirs('./data', exist_ok=True)

# 심볼릭 링크 생성 (이미 있으면 삭제 후 재생성)
link_path = './data/mit-bih-arrhythmia-database-1.0.0'
if os.path.islink(link_path):
    os.unlink(link_path)
elif os.path.exists(link_path):
    import shutil
    shutil.rmtree(link_path)

os.symlink(DRIVE_DATA_PATH, link_path)

# 데이터 확인
print("📂 데이터 경로:", link_path)
print("\n파일 목록:")
!ls -la {link_path} | head -20

# 레코드 수 확인
dat_files = [f for f in os.listdir(link_path) if f.endswith('.dat')]
print(f"\n✅ 총 {len(dat_files)}개 레코드 확인됨")



## 3️⃣ 설정 확인 및 수정


In [ ]:
# config.py 설정 확인
from config import (
    EXP_NAME, BATCH_SIZE, EPOCHS, LR, CLASSES,
    DS1_TRAIN, DS1_VALID, DS2_TEST, MODEL_CONFIG
)

print("=" * 60)
print("현재 설정")
print("=" * 60)
print(f"실험 이름: {EXP_NAME}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Learning Rate: {LR}")
print(f"Classes: {CLASSES}")
print(f"\nTrain records ({len(DS1_TRAIN)}): {DS1_TRAIN}")
print(f"Valid records ({len(DS1_VALID)}): {DS1_VALID}")
print(f"Test records ({len(DS2_TEST)}): {DS2_TEST}")
print(f"\nModel config: {MODEL_CONFIG}")


In [ ]:
# [선택] 설정 변경이 필요하면 여기서 수정
# config.py 파일을 직접 수정하거나 아래처럼 덮어쓰기

# 예시: 실험 이름 변경 (baseline, naive_concatenate, cross_attention 중 선택)
# !sed -i 's/EXP_NAME = .*/EXP_NAME = "cross_attention"/' config.py

# 예시: 에폭 수 변경 (빠른 테스트용)
# !sed -i 's/EPOCHS = .*/EPOCHS = 10/' config.py


## 4️⃣ 학습 실행


In [ ]:
# 학습 실행
!python main.py


In [ ]:
# 결과 폴더 확인
!ls -la ECG_Results/


In [ ]:
# 최신 실험 폴더 찾기
import glob

exp_dirs = sorted(glob.glob('ECG_Results/experiment_*'))
if exp_dirs:
    latest_exp = exp_dirs[-1]
    print(f"최신 실험 폴더: {latest_exp}")
    print("\n폴더 내용:")
    !ls -la {latest_exp}
    print("\n모델 가중치:")
    !ls -la {latest_exp}/best_weights/


In [ ]:
# 결과 Excel 파일 읽기
import pandas as pd
import glob

exp_dirs = sorted(glob.glob('ECG_Results/experiment_*'))
if exp_dirs:
    latest_exp = exp_dirs[-1]
    xlsx_files = glob.glob(f'{latest_exp}/*.xlsx')
    
    for xlsx in xlsx_files:
        print(f"\n{'='*60}")
        print(f"📊 {xlsx}")
        print("=" * 60)
        
        # Macro 시트
        df_macro = pd.read_excel(xlsx, sheet_name='Macro')
        print("\n[Macro Metrics]")
        print(df_macro.to_string(index=False))
        
        # Per-class 시트
        df_per_class = pd.read_excel(xlsx, sheet_name='Per_Class')
        print("\n[Per-Class Metrics]")
        print(df_per_class.to_string(index=False))


In [ ]:
# Confusion Matrix 이미지 표시
from IPython.display import Image, display
import glob

exp_dirs = sorted(glob.glob('ECG_Results/experiment_*'))
if exp_dirs:
    latest_exp = exp_dirs[-1]
    png_files = glob.glob(f'{latest_exp}/*.png')
    
    for png in png_files:
        print(f"\n{png}")
        display(Image(filename=png, width=500))


## 6️⃣ 결과를 Google Drive에 저장


In [ ]:
# Google Drive는 이미 마운트됨 (섹션 2에서 완료)
# 결과 저장 경로 확인
print("📂 결과 저장 경로: /content/drive/MyDrive/ECG_Results/")


In [ ]:
# 결과를 Google Drive로 복사
import shutil
import os
import glob

exp_dirs = sorted(glob.glob('ECG_Results/experiment_*'))
if exp_dirs:
    latest_exp = exp_dirs[-1]
    dest_path = f'/content/drive/MyDrive/ECG_Results/{os.path.basename(latest_exp)}'
    os.makedirs(dest_path, exist_ok=True)
    
    # best_weights 폴더 복사
    best_weights_src = os.path.join(latest_exp, 'best_weights')
    if os.path.exists(best_weights_src):
        dest_weights = os.path.join(dest_path, 'best_weights')
        if os.path.exists(dest_weights):
            shutil.rmtree(dest_weights)
        shutil.copytree(best_weights_src, dest_weights)
        print(f"✅ Best weights saved to: {dest_weights}")
    
    # xlsx 파일들 복사
    for f in glob.glob(f'{latest_exp}/*.xlsx'):
        shutil.copy(f, dest_path)
    for f in glob.glob(f'{latest_exp}/*.png'):
        shutil.copy(f, dest_path)
    
    print(f"✅ All results saved to: {dest_path}")


---
## 📝 빠른 테스트 (에폭 5회)


In [ ]:
# 빠른 테스트용 설정 변경
!sed -i 's/EPOCHS = .*/EPOCHS = 5/' config.py
!sed -i 's/EXP_NAME = .*/EXP_NAME = "cross_attention_test"/' config.py

# 변경 확인
!grep -E "^(EPOCHS|EXP_NAME)" config.py


In [ ]:
# 빠른 테스트 실행
!python main.py
